In [10]:
import os
import pandas as pd
import numpy as np
import cv2

In [8]:
# def json2df():
def cntarea(cnt):
    cnt = np.array(cnt)
    area = cv2.contourArea(cnt)
    return area

def cntperi(cnt):
    cnt = np.array(cnt)
    perimeter = cv2.arcLength(cnt,True)
    return perimeter

def cntMA(cnt):
    cnt = np.array(cnt)
    #Orientation, Aspect_ratio
    (x,y),(MA,ma),orientation = cv2.fitEllipse(cnt)
    return np.max((MA,ma)),np.min((MA,ma)),orientation

In [2]:
jsonsrc = r'\\10.99.68.178\andreex\data\monkey fetus\gestational 40\hovernet_out\json'
dst = os.path.join(os.path.dirname(jsonsrc), 'df')
if not os.path.exists(dst): os.mkdir(dst)

In [3]:
jsonnm = 'monkey_fetus_40_0301.json'
jsonpth = os.path.join(jsonsrc,jsonnm)


'monkey_fetus_40_0301.json'

In [4]:
imID,ext = os.path.splitext(os.path.basename(jsonpth))
dstfn = os.path.join(dst, '{}.pkl'.format(imID))
if os.path.exists(dstfn):
    print("pkl already exists, skipping the file ID {}".format(imID))
    # return

In [6]:
try:
    json = pd.read_json(jsonpth, orient='index')
except:
    print('cannot open json')

In [7]:
json = pd.DataFrame(json[0].loc['nuc']).T.drop(columns=['type_prob'])
json = json[json['contour'].map(len) > 5].reset_index(drop=True)

In [11]:
json['Area'] = json['contour'].apply(lambda row: cntarea(row))
json['Perimeter'] = json['contour'].apply(lambda row: cntperi(row))
json['Circularity'] = 4 * np.pi * json['Area'] / json['Perimeter'] ** 2
json['MA'] = json['contour'].apply(lambda row: cntMA(row))
json[['MA', 'ma', 'orientation']] = pd.DataFrame(json.MA.tolist())
json['AspectRatio'] = json['MA'] / json['ma']
json['imID'] = imID
json.to_pickle(dstfn)


In [12]:
json.head()

,bbox,centroid,contour,type,Area,Perimeter,Circularity,MA,ma,orientation,AspectRatio,imID
0,"[[1057, 12054], [1068, 12066]]","[1819.5, 11301.521739130434]","[[1817, 11297], [1817, 11298], [1816, 11299], ...",None,75.5,35.556349,0.750451,13.446315,8.539762,34.649132,1.574554,monkey_fetus_40_0301
1,"[[1088, 12123], [1097, 12134]]","[1887.6029411764705, 11331.970588235294]","[[1889, 11328], [1888, 11329], [1887, 11329], ...",None,54.0,30.142135,0.746888,11.154696,6.898702,62.998440,1.616927,monkey_fetus_40_0301
2,"[[1090, 12138], [1107, 12149]]","[1903.1530612244899, 11337.551020408164]","[[1906, 11330], [1905, 11331], [1904, 11331], ...",None,79.5,42.041630,0.565220,18.602440,6.091478,26.693090,3.053847,monkey_fetus_40_0301
3,"[[1381, 11969], [1405, 11985]]","[1737.0045662100456, 11632.383561643835]","[[1730, 11621], [1729, 11622], [1729, 11625], ...",None,191.5,62.526911,0.615524,25.748045,10.807999,151.067474,2.382314,monkey_fetus_40_0301
4,"[[1532, 12034], [1556, 12045]]","[1799.4916666666666, 11783.208333333334]","[[1795, 11772], [1795, 11773], [1794, 11774], ...",None,93.5,57.213203,0.358946,25.887810,5.386616,161.308823,4.805950,monkey_fetus_40_0301
